In [1]:
## Webcam image recognition
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import img_to_array

In [2]:
key = cv2. waitKey(1)
webcam = cv2.VideoCapture(0)
while True:
    try:
        check, frame = webcam.read()
        cv2.imshow("Capturing", frame)
        key = cv2.waitKey(1)
        if key == ord('s'): 
            cv2.imwrite(filename='Captured_image.jpg', img=frame)
            webcam.release()
            img_new = cv2.imread('Captured_image.jpg', cv2.IMREAD_GRAYSCALE)
            img_new = cv2.imshow("Captured Image", img_new)
            cv2.waitKey(1)
            cv2.destroyAllWindows()
        
            break
        elif key == ord('q'):
            print("Turning off camera.")
            webcam.release()
            print("Camera off.")
            print("Program ended.")
            cv2.destroyAllWindows()
            break
        
    except(KeyboardInterrupt):
        print("Turning off camera.")
        webcam.release()
        print("Camera off.")
        print("Program ended.")
        cv2.destroyAllWindows()
        break

In [3]:
## Webcam image recognition
detection_model_path = 'haarcascade_frontalface_default.xml'
image_path = 'Captured_image.jpg'

face_detection = cv2.CascadeClassifier(detection_model_path)

emotion_classifier = load_model("model_100epochs.h5")

emotions = ['angry', 'disgust', 'scared', 'happy', 'sad', 'surprise', 'neutral']


color_frame = cv2.imread(image_path)
gray_frame = cv2.imread(image_path,0)

cv2.imshow('image',color_frame)
    
detected_faces = face_detection.detectMultiScale(color_frame, scaleFactor=1.1, minNeighbors=5,minSize=(30,30), flags=cv2.CASCADE_SCALE_IMAGE)
print('Number of faces detected :',len(detected_faces))

if len(detected_faces)>0:
    detected_faces = sorted(detected_faces, reverse=True, key=lambda x: (x[2]-x[0])*(x[3]-x[1]))[0] ##If more than one faces
    (fx, fy, fw, fh) = detected_faces
    
    im = gray_frame[fy:fy+fh, fx:fx+fw]
    im = cv2.resize(im, (48,48)) ## The model is trained on 48*48 pixel image
    im = im.astype("float")/255.0
    im = img_to_array(im)
    im = np.expand_dims(im, axis=0)
    
    preds = emotion_classifier.predict(im)[0]
    emotion_probability = np.max(preds)
    label = emotions[preds.argmax()]
    
    cv2.putText(color_frame, label, (fx, fy-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45,(0,0,255),2)
    cv2.rectangle(color_frame, (fx,fy), (fx+fw, fy+fh),(0,0,255),2)
    

cv2.imwrite('output_test_for_camera_image.jpg',color_frame)
cv2.imshow('emotion detected',color_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

Number of faces detected : 1
